In [1]:
from pathlib import Path
import numpy as np
import torch, torchvision, json
import torchvision.transforms as transforms
from torch.nn import CrossEntropyLoss
# from efficientnet_pytorch import EfficientNet as EN
from tqdm import tqdm
import torch.nn as nn
from utils import *
device="cuda"

In [9]:
model = torchvision.models.wide_resnet50_2(pretrained=True)

In [2]:
data_dir = Path('tiny-imagenet-200')
data_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.4805, 0.4483, 0.3978), (0.263, 0.257, 0.267)),
])

In [9]:
valid_set = torchvision.datasets.ImageFolder(data_dir / 'val', data_transforms)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=32,
                                           shuffle=False, num_workers=4, pin_memory=True)
train_set = torchvision.datasets.ImageFolder(data_dir / 'train', data_transforms)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32,
                                           shuffle=True, num_workers=4, pin_memory=True)
loss = CrossEntropyLoss()
lowest_train_label = train_set[-1][1] - 199
lowest_val_label = valid_set[-1][1] - 199

In [ ]:
loader = iter(valid_loader)
x, y = next(loader)

In [15]:
y

tensor([1])

In [8]:
experiment = 303
seed = 303
model_id = f"{experiment}_{seed}"
model_path = Path(f"outputs/experiment_{experiment}/data/{model_id}").rglob('*pt')
model_path = list(model_path)[0]
model = torchvision.models.wide_resnet50_2(pretrained=True)
model.fc = nn.Linear(2048, 200)
model.load_state_dict(torch.load(model_path))
model.to(device)
print(model_path, 'loaded')

outputs/experiment_303/data/303_303/model_2_val_acc=0.8041.pt loaded


In [12]:
for i in [63, 62, 61, 60, 58, 57, 56, 54, 52, 51, 49, 46, 45, 44, 
          43, 42, 37, 35, 34, 33, 32, 31, 29, 28, 27, 26, 25, 22,
          21, 20, 19, 18, 17, 16, 14, 12, 11, 10, 6, 3, 1]:
    TuckerStructured(model.conv1, name='weight', amount=0, dim=0,filt=i)

In [10]:
def valid(model, dataset, dataloader, loss, device, label):
    running_loss = 0.0
    right = 0
    total = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader),
                            total=len(dataset) / dataloader.batch_size):
            data, y = data
            data = data.to(device)
            y = y.to(device) - label
            ans = model.forward(data)
            los = loss(ans, y)
            running_loss += los.item()

                
            right += torch.sum(torch.eq(torch.argmax(ans, dim=1), y))
            total += y.shape[0]
    return running_loss, running_loss / len(dataloader.dataset), right/total

In [28]:
# train_loss, avg_train, train_acc = valid(model, train_set, train_loader, loss, device)
# # print(train_loss, avg_train)

In [13]:
valid_loss, avg_valid, valid_acc = valid(model, valid_set, valid_loader, loss, device, lowest_val_label)
print(valid_loss, avg_valid, valid_acc)

100%|██████████| 313/312.53125 [01:19<00:00,  3.96it/s]

2007.4560351371765 0.2007255309606216 tensor(0.0879, device='cuda:0')


In [17]:
model.conv1.weight_mask

tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
        

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.1,
                                    momentum=0.1, weight_decay=0.1)
sched = ReduceLROnPlateau(optimizer, factor=0.5, patience=5)
loss = Loss(F.cross_entropy)._loss_fn
trainer = create_supervised_trainer(model, optimizer, loss,
                                            device=device)